In [13]:
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin, urldefrag

In [3]:
urls=set()
driver = webdriver.Chrome()

In [5]:
def extract_urls(url):
    urls.add(f"{url}")
    print(url)
    driver.get(url)
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    for aTag in soup.find_all('a'):
        href = aTag.get('href')
        if href is None:
            continue
        if href.startswith("http"):
            urls.add(href)
            continue
        elif href.startswith("#"):
            continue
        elif href.startswith("mailto:") :
            urls.add(href)
        else:
            link = f"https://msrit.edu/{href}"
            if link.startswith("../"):
                link=link[3:]
            if link in urls:
                continue
            else:
                extract_urls(link)

In [6]:
extract_urls("https://msrit.edu")

https://msrit.edu
https://msrit.edu/admissions.html
https://msrit.edu/index.html
https://msrit.edu/about-us.html
https://msrit.edu/about-us.html#history
https://msrit.edu/governance.html
https://msrit.edu/governance.html#bom
https://msrit.edu/gov-others.html
https://msrit.edu/admissions.html#po
https://msrit.edu/admissions.html#research-programs
https://msrit.edu/admissions.html#prospectus
https://msrit.edu/admissions.html#fee
https://msrit.edu/admissions.html#scholarship
https://msrit.edu/admissions.html#contact
https://msrit.edu/department/architecture.html
https://msrit.edu/../index.html
https://msrit.edu/department/ai_ds.html
https://msrit.edu/../about-us.html
https://msrit.edu/department/ai_ml.html
https://msrit.edu/../about-us.html#history
https://msrit.edu/department/biotechnology.html
https://msrit.edu/../governance.html
https://msrit.edu/department/chemical-engineering.html
https://msrit.edu/../governance.html#bom
https://msrit.edu/department/chemistry.html
https://msrit.edu/.

In [7]:
with open('urls.json', 'w') as f:
    json.dump(list(urls), f)

In [11]:
clean_urls = set(urls)
print(len(clean_urls))
for url in urls:
    if "../" in url:
        clean_urls.add("https://msrit.edu/"+url.split("../")[1])
        clean_urls.remove(url)
print(len(clean_urls))

1550
1462


In [14]:
# Defragment all the urls in the set
defragmented_urls = set()
for url in clean_urls:
    defragmented_urls.add(urldefrag(url)[0])
print(len(defragmented_urls))

1425


In [18]:
msrit_urls = [url for url in defragmented_urls if "https://msrit.edu/" in url]

In [19]:
# Use requests library to get the status code of each url and store valid URLs in valid_msrit_urls
valid_msrit_urls = []
for url in msrit_urls:
    response = requests.get(url)
    if response.status_code == 200:
        valid_msrit_urls.append(url)

In [20]:
with open('valid_msrit_urls.json', 'w') as f:
    json.dump(valid_msrit_urls, f)

In [21]:
external_urls = [url for url in defragmented_urls if "https://msrit.edu/" not in url]
with open('external_urls1.json', 'w') as f:
    json.dump(external_urls, f)